# Performance in Python

## 1. Introduction

Python is a high-level language designed to write code fast, more than write fast code. In such high-level languages, the bulk of your computation should be performed by off-the-shelf libraries, with your own code only tying everything together, or aspects specific to your application. This is also important for performance, as if you find yourself doing a low-level computation in plain Python, you're probably making your program way slower than it could be, and wasting time reinventing the wheel along the way.

We can ask the python notebook to measure the execution time of a statement using `%timeit` (notebook [magic](https://ipython.readthedocs.io/en/stable/interactive/magics.html) for [`timeit`](https://docs.python.org/3/library/timeit.html)). The statement will not be executed just one, but an appropriately large number of times, and generally gives a good measurement of its running time.
To benchmark deeper or more complex programs, we can use profiling tools such as [cProfile](https://docs.python.org/3/library/profile.html). There exists great modules for visualizing the output of the profiler, such as [SnakeViz](https://jiffyclub.github.io/snakeviz/).

## 2. Built-ins

In Python, we typically use [built-in functions](https://docs.python.org/3/library/functions.html) and libraries (modules) that are well-optimized for specific tasks, such as `len`, `sum`, `max` and the like. You should use them when possible, rather than your own implementations. We will see later that in some cases they can be superseded by module-specific functions (like `numpy`'s functions on `numpy`'s objects, if you're doing math), but compared to general Python, built-ins are very fast.


Let's compare summing a list of integers with the built-in `sum` and with the usual, naive `for` loop:

In [1]:
import random

numbers = [random.randint(0,10) for i in range(1_000_000)]

def my_sum(numbers):
    s = 0
    for n in numbers:
        s += n
    return s

print("Sum via for loop:")
%timeit my_sum(numbers)
print("Sum via builtin:")
%timeit sum(numbers)


Sum via for loop:
72.1 ms ± 879 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Sum via builtin:
12.8 ms ± 367 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


We get a speedup of roughly 5x!

## 3. List comprehensions

We generated the random array of numbers using a *list comprehension*. This is a way to build lists without explicit `for` loops, using a generating expression. They are usually significantly faster than iteratively building a list using append. Let's measure that, by squaring a list of numbers. Along the way, we will also compare iterating the list using the keyword `in` against explicitly indexing elements.

In [2]:
def square_list_direct_indexing(numbers):
    result = []
    for i in range(len(numbers)):
        result.append(numbers[i]*numbers[i])
    return result

def square_list_using_in(numbers):
    result = []
    for n in numbers:
        result.append(n*n)
    return result

%timeit square_list_direct_indexing(numbers)
%timeit square_list_using_in(numbers)
%timeit [n*n for n in numbers]


220 ms ± 1.53 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
129 ms ± 2.98 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
59.4 ms ± 709 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


As you can see, we are roughly getting a 2x speedup each time, for a total of 4x.

For squaring, you might be tempted to use the `**` exponentiation operator, especially if your variable names are long and multiplying `variable * variable` is tedious. But surprise, the exponentiation operator and the built-in `pow` are more expensive (probably because much more general) than multiplication. `pow` also uses float semantics systematically, whereas `**` can exponentiate integers directly. See:

In [3]:
%timeit [pow(n, 2) for n in numbers]
%timeit [n**2 for n in numbers]
%timeit [n*n for n in numbers]

387 ms ± 1.09 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
414 ms ± 490 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)
59.9 ms ± 391 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


## 4. Function call overhead

Function calls also have significant overhead in Python. Let's compare calling a function inside a `for` comprehension, against putting the `for` comprehension inside the function:

In [4]:
from math import sqrt

def my_function(x):
    return 0.5 * sqrt(1 + x*x)

def my_function_on_arrays(xs):
    return [0.5 * sqrt(1 + x*x) for x in xs]

%timeit [my_function(x) for x in numbers]
%timeit my_function_on_arrays(numbers)

361 ms ± 1.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
256 ms ± 1.92 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


As you can see, avoiding calls in a loop saves significant time.

## 5. Data structures and membership testing

We will now highlight the importance of tailoring which data structure you use to the type of computation you are going to do.

In [5]:
# Load all english words in alphabetic order
with open("words.txt", "r") as f:
    words = f.read().splitlines()
    words = [word.lower() for word in words]

print(len(words))

466550


In [6]:
# Position of element in list matters for "in"
print("Looking for 'anaconda' and 'zebra' in the word list:")
%timeit "anaconda" in words
%timeit "zebra" in words


# Let's spend some effort converting that list into a set, and test membership on that
print("Converting the list to a set:")
wordset = set(words)
%timeit set(words)
print("Looking for 'anaconda' and 'zebra' in the word set:")
%timeit "anaconda" in wordset
%timeit "zebra" in wordset


Looking for 'anaconda' and 'zebra' in the word list:
350 µs ± 2.14 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
12.2 ms ± 162 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
Converting the list to a set:
67.1 ms ± 1.51 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
Looking for 'anaconda' and 'zebra' in the word set:
67.6 ns ± 0.293 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)
66.7 ns ± 0.484 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


We can either spend anywhere between 0 and >12ms per test on a list with a O(n) sequential scan, or take ~66ms to construct a set and then have ≤100ns per lookup. For this collection of half a million short strings, it becomes worthwhile at about 10 tests! 

Conversion to set is even more worthwhile when the base list contains duplicates, which was not the case here.

The takeaway is that in Python, the more straightforward you are, generally the faster your code runs. This is not necessarily the case, for example, in compiled languages like C/C++ where a (very) smart compiler can figure out optimizations for you. In Python, there is no advanced compiler to help you, so the more specific you are when writing your code, the faster it can be executed (we will see later that with Numba and JIT compilation, we can sometimes have the best of both worlds). 

Just as in any language, using [the right data structure](https://docs.python.org/3/library/collections.html#module-collections) for the usage you're making is very important.





In the next section, we will illustrate how using-the-shelf specialized modules whenever relevant can save a lot of time, both writing *and* executing code, with the example of `numpy`.

## 6. Arrays with NumPy

For numerical & scientific computing, using modules such as [**NumPy**](https://numpy.org/), which at their core use well-optimized C code, can lead to significant speed-ups.

NumPy is designed for [array programming](https://en.wikipedia.org/wiki/Array_programming), which supports vectorized operations that operate on entire arrays at once. When using such libraries, vectorized operations can lead to massive speed-ups over nested for-loops.

For more information on array programming, check out [this tutorial](https://realpython.com/numpy-array-programming/) and [NumPy's documentation](https://numpy.org/doc/stable/).

First, let's compare NumPy arrays to Python lists. We will use a list of random numbers, and compare how fast squaring each entry is with lists, and with Numpy arrays:

In [7]:
import numpy as np

numbers = [random.random() for i in range(1_000_000)]
numbers_arr = np.array(numbers)

%timeit [n*n for n in numbers]
%timeit numbers_arr ** 2

116 ms ± 892 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
906 µs ± 22.9 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


As you can see, NumPy arrays are over 100x faster for this specific task!

Now, let's show how fast vectorized operations are over nested "for-loops". 

To show that, let's implement a function that multiplies each entry of a matrix by its row number.

In [8]:
# Naive implementation that iterates over each entry of the array
def row_multiply_naive(x):
    x = x.copy()
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            x[i, j] = x[i, j] * i
    return x

# Slightly better naive implementation, that only iterates over rows
def row_multiply_naive_2(x):
    x = x.copy()
    for i in range(x.shape[0]):
        x[i, :] = x[i, :] * i
    return x

# Vectorized implementation
def row_multiply_vectorized(x):
    return x * np.arange(x.shape[0])[:, np.newaxis]

In [9]:
# Initialize a random matrix with 10'000 rows and 100 columns
a = np.random.rand(10_000, 100)

# Check that all three implementations give the same result
assert(np.all(row_multiply_naive(a) == row_multiply_naive_2(a)))
assert(np.all(row_multiply_naive(a) == row_multiply_vectorized(a)))

# Benchmark implementations
print("Naive:")
%timeit row_multiply_naive(a)

print("Naive 2:")
%timeit row_multiply_naive_2(a)

print("Vectorized:")
%timeit row_multiply_vectorized(a)

Naive:
920 ms ± 8.58 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Naive 2:
32.9 ms ± 319 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)
Vectorized:
2.1 ms ± 16.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Using Numpy's vectorization results in a roughly 400x speed-up over the naive implementation!

## 7. JIT compilation with Numba

In [ ]:
# Run this cell if you don't have Numba installed
!pip install numba

[Numba](https://numba.pydata.org/) is a module that allows you to significantly speed up functions, by translating Python functions to optimized machine code at runtime, using Just-in-Time compilation (JIT). The Python language has characteristics that make it poorly suited for static optimization, but with the magic of JIT, Numba can compile your code knowing some extra information that cannot always be known statically.

To do so, add the `@jit` decorator to a function, which will let Numba optimize it, although keep in mind that not all functions are supported by Numba. You can find more about what kind of functions should be compiled by Numba [here](https://numba.readthedocs.io/en/stable/user/troubleshoot.html). 

If you want to learn more about Numba, check out the [quick-start guide](https://numba.readthedocs.io/en/stable/user/5minguide.html).

**Example 1:** (Very naively) counting the number of multiples

In [10]:
from numba import jit

# Function that (very naively) counts the number of integers between 0 and 10 million that are divisible by a given integer
def count_multiples_naive(divisor):
    count = 0
    for i in range(10_000_000):
        if i % divisor == 0:
            count += 1
    return count

# Same function, but with the jit decorator added
@jit
def count_multiples_naive_jit(divisor):
    count = 0
    for i in range(10_000_000):
        if i % divisor == 0:
            count += 1
    return count

In [11]:
assert(count_multiples_naive(42) == count_multiples_naive_jit(42))

print("Naive implementation:")
%timeit count_multiples_naive(42) 
print("Naive implementation with Numba JIT compilation:")
%timeit count_multiples_naive_jit(42)

Naive implementation:
918 ms ± 7.62 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
Naive implementation with Numba JIT compilation:
41.2 ms ± 134 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


With the addition of just one `@jit` annotation, we obtain a roughly 20x speed-up!

**Example 2:** Estimating pi using the Monte Carlo method

In [12]:
import random 

# Estimating pi using the Monte-Carlo method
def monte_carlo_pi(n_samples):
    acc = 0
    for i in range(n_samples):
        x = random.random()
        y = random.random()
        if (x ** 2 + y ** 2) < 1.0:
            acc += 1
    return 4.0 * acc / n_samples


@jit
def monte_carlo_pi_jit(n_samples):
    acc = 0
    for i in range(n_samples):
        x = random.random()
        y = random.random()
        if (x ** 2 + y ** 2) < 1.0:
            acc += 1
    return 4.0 * acc / n_samples



In [13]:
# Timing 1 million samples
%timeit monte_carlo_pi(1_000_000)
%timeit monte_carlo_pi_jit(1_000_000)

586 ms ± 16.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
19.5 ms ± 58.8 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


We obtain a roughly 30x speed-up!

To apply what you have learned in this document, check out the exercises, and try to obtain those sweet sweet speedups using the tricks you just learned! Gotta go fast!